<a href="https://colab.research.google.com/github/tjdux/ML/blob/main/10_cross_validation_and_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 검증 세트 (validation set)
- 하이퍼파라미터 값을 바꾼 여러 모델을 많이 만들어서 테스트 세트로 평가하면 결국 테스트 세트에 잘 맞는 모델이 만들어지는 것
- 테스트 세트로 일반화 성능을 올바르게 예측하려면 테스트 세트는 최종 모델을 선택할 때까지 사용하지 말아야 함
- 검증 세트
  - 훈련 세트를 또 나눈 세트
  - 보통 훈련 세트의 20 - 30%를 떼어내어 검증 세트
- 검증 세트를 이용한 모델 학습 방법
  1. 훈련 세트로 모델을 훈련하고 검증 세트로 모델을 평가하여 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고름
  2. 이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련
  3. 마지막에 테스트 세트에서 최종 점수를 평가

In [41]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [42]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [43]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [44]:
# train_input, train_target을 다시 나누어 훈련 세트 sub_input, sub_target과 검증 세트 val_input, val_target으로 만듦
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [45]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [46]:
# sub_input, sub_target을 이용해 모델 학습 후 val_input, val_target으로 평가
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42);
dt.fit(sub_input, sub_target);
print(f"훈련 세트 점수: {dt.score(sub_input, sub_target)}")
print(f"검증 세트 점수: {dt.score(val_input, val_target)}")

훈련 세트 점수: 0.9971133028626413
검증 세트 점수: 0.864423076923077


- 과대 적합

## 교차 검증 (cross validation)
- 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복
- 이 점수를 평균하여 최종 검증 점수
- 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터 사용 가능
- k-폴드 교차 검증: 훈련 세트를 몇 부분으로 나누냐에 따라 다르게 부름
- 보통 5-폴드 교차 검증이나 10-폴드 교차 검증을 많이 사용
  - 데이터의 80-90%까지 훈련에 사용 가능
  - 검증 세트가 줄어들지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수

In [47]:
# cross_validate(): 교차 검증 함수, 평가할 모델과 훈련 세트 전체를 매개변수로 전달
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.00927949, 0.00857115, 0.00876617, 0.00844932, 0.00791836]),
 'score_time': array([0.00140238, 0.00115848, 0.00127959, 0.00116611, 0.00114441]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

- fit_time: 모델을 훈련하는 시간
- score_time: 검증하는 시간
- 각 키마다 5개의 숫자 ➡️ 기본적으로 5-폴드 교차 검증 수행
- 교차 검증의 최종 점수는 test_score의 평균

In [48]:
# 교차 검증 최종 점수
import numpy as np
np.mean(scores['test_score'])

np.float64(0.855300214703487)

- ⚠️ cross_validate()는 훈련 세트를 섞어서 폴드를 나누지 않음
  - 위의 예제에서는 train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트로 나눈 것이기 때문에 따로 섞을 필요 없음
  - 교차 검증을 할 때 훈련 세트를 섞으려면 분할기 (splitter)를 지정해야 함
- 분할기 (splitter)
  - 교차 검증에서 폴드를 어떻게 나눌지 결정
  - cross_validate() 함수의 경우
    - 회귀 모델일 경우 KFold 분할기 사용
    - 분류 모델일 경우 StraitifiedKFold 사용

In [49]:
# 분류 모델이므로 StratifiedKFold 사용
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
np.mean(scores['test_score'])

np.float64(0.855300214703487)

In [50]:
# 훈련 세트를 섞은 후 10-폴드 교차 검증 수행
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

np.float64(0.8574181117533719)

## 하이퍼파라미터 튜닝
- 파라미터
  - 모델 파라미터: 머신러닝 모델이 학습하는 파라미터
  - 하이퍼파라미터: 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터
  - 파라미터는 모두 클래스나 메서드의 매개변수로 표현
- 하이퍼파라미터 튜닝 과정
  1. 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련
  2. 검증 세트의 점수나 교차 검증을 통해 매개변수를 조금씩 바꾸기
  3. 매개변수를 바꿔가면서 모델을 훈련하고 교차 검증을 수행
- ⭐ 하이퍼파라미터 튜닝에서 한 매개변수의 최적값을 찾고, 그 다음에 다른 매개변수의 최적값을 찾는 것은 바람직하지 않음


### 그리드 서치 (Grid Search)
- 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행
- 그리드 서치 과정
  1. 탐색할 매개변수 지정
  2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는  매개변수 조합을 찾기. 그 조합은 그리드 서치 객체에 저장
  3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델을 훈련. 이 모델도 그래드 서치에 저장

#### min_impurity_decrease 매개변수의 최적값 찾기
- min_impurity_decrease: 노드를 분할하기 위한 불순도 감소 최소량 지정

In [51]:
# 결정 트리 모델에서 min_impurity_decrease 매개변수의 최적값 찾기
from sklearn.model_selection import GridSearchCV

params = {"min_impurity_decrease": [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

- GridSearchCV의 cv 매개변수 기본값은 5 ➡️ 5-폴드 교차 검증 수행
  - 위의 예제에서는 5 * 5 = 25개의 모델 훈련
- 매개변수 n_jobs
  - 병렬 실행에 사용할 CPU 코어 수 지정
  - -1: 시스템에 있는 모든 코어 사용

In [52]:
# best_estimator_: 검증 점수가 가장 높은 모델의 매개변수 조합으로 다시 전체 훈련 세트로 훈련한 모델
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.9615162593804117

In [53]:
# best_params_: 최적의 매개변수
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [54]:
# cv_results 속성의 mean_test_score: 각 매개변수에서 수행한 교차 검증의 평균 점수
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [55]:
# np.argmax(): 가장 큰 값의 인덱스 반환
best_index = np.argmax(gs.cv_results_['mean_test_score'])
# 평균 점수가 가장 좋은 매개변수 값 출력
gs.cv_results_['params'][best_index]

{'min_impurity_decrease': 0.0001}

#### min_impurity_decrease, max_depth, min_samples_split 매개변수의 최적값 찾기
- min_impurity_decrease: 노드를 분할하기 위한 불순도 감소 최소량 지정
- max_depth: 트리의 깊이 제한
- min_samples_split: 노드를 나누기 위한 최소 샘플 수

In [56]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)}

In [57]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [58]:
# 최상의 매개변수 조합
gs.best_params_

{'max_depth': 14,
 'min_impurity_decrease': np.float64(0.0004),
 'min_samples_split': 12}

In [59]:
# 최상의 교차 검증 점수
np.max(gs.cv_results_['mean_test_score'])

np.float64(0.8683865773302731)

### 랜덤 서치 (Random Search)
- 매개변수의 값이 수치형이고 연속적인 실숫값일 때, 너무 많은 매개 변수 조건이 있을 때 사용
- 매개변수 값의 목록을 전달하는 것이 아닌 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달
- 그리드 서치보다 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색 가능

#### 확률 분포 샘플링
- uniform, randint 클래스
  - 주어진 범위에서 고르게 값을 뽑음
  - 균등 분포에서 샘플링
  - uniform은 실숫값을, randint는 정숫값을 뽑음

In [60]:
# radint 샘플링
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([8, 1, 7, 0, 6, 0, 9, 5, 6, 2])

In [61]:
# 1000개를 샘플링하여 각 숫자의 개수를 세기
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([102,  86, 102,  97,  78, 121,  78, 114, 109, 113]))

In [62]:
# uniform 샘플링
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.25599315, 0.8842684 , 0.69800401, 0.15838913, 0.72662412,
       0.47051374, 0.7902851 , 0.85070623, 0.75456937, 0.81290653])

#### min_impurity_decrease, max_depth, min_samples_split, min_samples_leaf 매개변수의 최적값 찾기
- min_impurity_decrease: 노드를 분할하기 위한 불순도 감소 최소량 지정
- max_depth: 트리의 깊이 제한
- min_samples_split: 노드를 나누기 위한 최소 샘플 수
- min_samples_leaf: 리프 노드가 되기 위한 최소 샘플의 개수

In [63]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [64]:
from sklearn.model_selection import RandomizedSearchCV

# n_iter 매개변수: 샘플링 횟수
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a3019f39290>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7a3019f38f10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a3019f39f90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a3019f399d0>},
                   random_state=42)

In [65]:
# 최적의 매개변수 조합
gs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': np.float64(0.00034102546602601173),
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [66]:
# 최고의 교차 검증 점수
np.max(gs.cv_results_['mean_test_score'])

np.float64(0.8695428296438884)

In [67]:
# best_estimator_: 최적의 모델
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.8928227823744468

## 확인 문제 풀이

In [68]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42, splitter='random'), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a3019f39290>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7a3019f38f10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a3019f39f90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a3019f399d0>},
                   random_state=42)

In [69]:
np.max(gs.cv_results_['mean_test_score'])

np.float64(0.8458726956392981)

In [70]:
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.8043101789493938